In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("Nat_Gas.csv")
df["Month"] = pd.to_datetime(df["Dates"], format='%m/%d/%y').dt.month
df["Year"] = pd.to_datetime(df["Dates"], format='%m/%d/%y').dt.year
df["Day"] = pd.to_datetime(df["Dates"], format='%m/%d/%y').dt.day
end_day = {}
end_day[1] = 31
end_day[2] = 28
end_day[3] = 31
end_day[4] = 30
end_day[5] = 31
end_day[6] = 30
end_day[7] = 31
end_day[8] = 31
end_day[9] = 30
end_day[10] = 31
end_day[11] = 30
end_day[12] = 31

/var/folders/63/jgs1r5x966xcvf9q4_wbmp940000gn/T/ipykernel_35617/702642609.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Month"] = pd.to_datetime(df["Dates"], format='%m/%d/%y').dt.month
/var/folders/63/jgs1r5x966xcvf9q4_wbmp940000

In [4]:
# Create the regression model for each month
Parameters = {}
for mon in range(1,13):
    df_month = df[df["Month"] == mon]
    Model = LinearRegression()
    X = df_month["Year"]
    Y = df_month["Prices"]
    Model.fit(X.values.reshape((-1,1)), Y.values)
    Parameters[mon] = [Model.intercept_, Model.coef_]

In [5]:
def PriceForecast1(Date):
    if isinstance(Date, str):
        DateTime = datetime.strptime(Date, "%m/%d/%y")
    elif isinstance(Date, datetime):
        DateTime = Date
    Month = DateTime.month
    Year = DateTime.year
    Day = DateTime.day
    Price_1 = Parameters[Month][0] + Parameters[Month][1][0]*Year
    end_day1 = end_day[Month]
    if Month == 1:
        Price_2 = Parameters[12][0] + Parameters[12][1][0]*(Year-1)
        #end_day2 = end_day[12]
    else:
        Price_2 = Parameters[Month-1][0] + Parameters[Month-1][1][0]*Year
        #end_day2 = end_day[Month-1]
    # If leap year
    if Year%4 == 0:
        if Month == 2:
            end_day1 = 29
        elif Month == 3:
            end_day2 = 29
    price_forecast = (Price_1*Day + Price_2*(end_day1 - Day))/end_day1
    return price_forecast

# Create a prototype pricing model

In [6]:
inj_amt = 1
def ContractPrice(Inj_dates, With_dates, Price, Inj_rate, Max_Vol, Strg_Cost):
    #--------------------------------------------------------------
    # Variables:
    # Inj_Dates: Injection dates is an array of dates when commodity will be injected in the storage unit
    # With_dates: Withdrawal dates would be the array of dates when commodity would be taken out from the storage unit
    # Price: Price is the array of prices of the commodity at those injection/withdrawal dates. 
    # It will be assumed that the injected commodity will be bought at the same price and withdrawal amount would be sold at the price that day
    # Max_Vol: The maximum volume of the storage unit. This will determine the quantity of commodity bought/sold
    # Strg_Cost: The storage cost of the commodity per day
    #--------------------------------------------------------------
    # Assumptions:
    # 1) Since no array is given for the amount of commodity injected, it will be assumed that the amount of commodity injected/withdrawn every time is a constant
    # 2) Assume no initial commodity in the storage unit. So any withdrawn date before the injection date would not do anything
    # 3) The storage costs is given in per day cost, irrespective of the amount stored
    In_tank = 0
    Contract_price = 0
    Cost = 0
    Sell = 0
    port_inj = pd.DataFrame([Inj_dates, np.ones((len(Inj_dates)))], columns=["Dates", "Action"])
    port_with = pd.DataFrame([With_dates, -np.ones((len(With_dates)))], columns=["Dates", "Action"])
    port = pd.concat([port_inj, port_with], axis=0)
    port['date_column'] = pd.to_datetime(df["Dates"])
    port_sorted = port.sort_values(by="date_column")
    for i in range(len(port_sorted)):
        if In_tank + port_sorted["Action"].iloc[i] >= 0 and In_tank + port_sorted["Action"].iloc[i] <= Max_Vol:
            In_tank += In_tank + port_sorted["Action"].iloc[i]
            Cost += Inj_rate
            if port_sorted["Action"].iloc[i] == 1:
                # Commodity injected in the storage tank
                Cost += PriceForecast1(port_sorted['date_column'].iloc[i])
                if i > 0 and In_tank > 0:
                    # Take into account the storage cost since the previous period
                    m = 1
                    while i-m >= 0 and port_sorted["Action"].iloc[i-m] == 0:
                        m += 1
                    delta_day = (port_sorted['date_column'].iloc[i] - port_sorted['date_column'].iloc[i-m]).days
                    Cost += Strg_Cost*delta_day
            elif port_sorted["Action"].iloc[i] == -1:
                # Commodity withdrawn from the storage tank and sold
                Sell += PriceForecast1(port_sorted['date_column'].iloc[i])
                if i > 0 and In_tank >= 0:
                    m = 1
                    while i-m >= 0 and port_sorted["Action"].iloc[i-m] == 0:
                        m += 1
                    delta_day = (port_sorted['date_column'].iloc[i] - port_sorted['date_column'].iloc[i-m]).days
                    Cost += Strg_Cost*delta_day
        else:
            port_sorted["Action"].iloc[i] = 0
    Contract_price = Sell - Cost
    return Contract_price

                